## READ ME
- ランダム行列の生成は$A = \begin{pmatrix}
U(-h,h) & U(-h,h) \\
U(-h,h) & U(-h,h)
\end{pmatrix}$で$A$を生成し，$X = A {}^t\!A$として正定値行列を生成

## ライブラリのimport

In [94]:
# !pip install mip 
from mip import Model, maximize, minimize, xsum
import numpy as np
from numpy import pi
from numpy.random import rand, randn
from numpy.linalg import solve, det
from scipy.linalg import sqrtm
import math


## 定数の設定

In [95]:
p = 2 # 行列の次元
N = 100 # データ数
lam = 0.01 # 正則化パラメータ
delta = 10.0
epsilon = 1.0
h = 2.0 #U(-h,h)

## Wasserstein距離を計算する関数

In [96]:
def wasserstein(x, y):
    z = sqrtm(sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2 * np.trace(z)
    return res

## Entropy正則化されたWasserstein距離を計算する関数

In [97]:
def reg_wasserstein(x, y):
    lam = 0.1 
    M_lam = sqrtm(lam * lam * np.eye(p) + sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2.0 * np.trace(M_lam) - 2.0 * lam * math.log(det(M_lam - lam * np.eye(p)),math.e) - 2.0 * lam * p * math.log(2.0 * math.pi * lam, math.e) - 4.0 * lam * p * math.log(2.0 * math.pi, math.e) - 2.0 * lam * p + 2.0 * lam * math.log(det(x @ y),math.e) + 4.0 * lam * p * (math.log(2.0 * math.pi, math.e) + 1.0)
    return res

In [98]:
## test
# p = 2
print(wasserstein(np.eye(p),np.eye(p)))

0.0


In [99]:
print(reg_wasserstein(np.eye(p),np.eye(p)))

0.6058665937452075


## 乱数の生成と距離グラフの生成

### wasserstein

In [100]:
P = []
nbh_w = [[] for _ in range(N)]

i = 0
np.random.seed(123)
while i < N:
    # rand_a = rand(3,1)
    a = np.array([[rand(1) * 2.0 * h - h, rand(1) * 2.0 * h - h],[rand(1) * 2.0 * h - h, rand(1) * 2.0 * h * h]])
    a = a.reshape((2,2)) 
    v = a @ a.T
    # B(I,delta)内になければskip
    if det(v) < 0.0001:
        continue 
    if wasserstein(np.eye(p),v) > delta :
        continue
    
    for j in range(0,i):
        w_dist = wasserstein(P[j],v)
        if w_dist < epsilon:
            nbh_w[i].append(j)
            nbh_w[j].append(i)
            
    # print(i)    
    nbh_w[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_w)
print(len(P),len(nbh_w))
    

[[0, 3, 10, 18, 30, 51, 60], [1, 14], [2, 7, 11, 17, 30, 38, 39, 60, 91], [0, 3, 18, 30], [4, 7, 23, 24, 30, 37, 53, 68, 69, 70], [5, 29, 34, 47, 52, 62, 73], [6, 10, 12, 26, 45, 86, 99], [2, 4, 7, 30, 31, 39, 54, 58, 93], [8, 12, 24, 33, 44, 45, 54, 67, 70, 71, 87], [9, 13, 29, 47, 73, 76, 85, 90], [0, 6, 10, 24, 26, 51, 82, 86], [2, 11, 17, 21, 38, 39, 61, 84, 91], [6, 8, 12, 24, 30, 45, 66, 67, 70, 71], [9, 13, 22, 25, 27, 47, 59, 73, 76, 81, 85, 88, 89, 90, 94], [1, 14, 38, 42, 52, 81], [15, 28, 75, 95, 99], [16, 21, 61, 64, 80, 93], [2, 11, 17, 38, 41, 60, 77, 91], [0, 3, 18], [19, 27, 33, 43, 44, 46, 54, 58, 59, 67, 72, 74, 81, 88, 89, 94, 98], [20, 99], [11, 16, 21, 39, 61, 80, 91], [13, 22, 25, 27, 47, 59, 73, 76, 81, 88, 89, 94], [4, 23, 37, 58, 63, 68, 97], [4, 8, 10, 12, 24, 30, 45, 53, 68, 69, 70, 82, 86], [13, 22, 25, 27, 47, 59, 73, 76, 85, 88, 89, 90, 94, 96], [6, 10, 26, 50, 51, 79, 99], [13, 19, 22, 25, 27, 43, 46, 47, 58, 59, 72, 81, 88, 89, 90, 94, 98], [15, 28, 34, 

### regularized wasserstein

In [101]:
P = []
nbh_rw = [[] for _ in range(N)]


i = 0
np.random.seed(123)
while i < N:
    # rand_a = rand(3,1)
    a = np.array([[rand(1) * 2.0 * h, rand(1) * 2.0 * h - h],[rand(1) * 2.0 * h - h, rand(1) * 2.0 *h]])
    a = a.reshape((2,2)) 
    v = a @ a.T
    # B(I,delta)内になければskip
    if reg_wasserstein(np.eye(p),v) > delta:
        continue
    
    for j in range(0,i):
        rw_dist = reg_wasserstein(P[j],v)
        if rw_dist < epsilon:
            nbh_rw[i].append(j)
            nbh_rw[j].append(i)
            
    # print(i)    
    nbh_rw[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_rw)
print(len(P),len(nbh_rw))
    

[[0], [1], [2], [3], [4], [5], [6], [7], [8, 78], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18, 43, 67, 82], [19], [20], [21, 59, 90, 95], [22], [23], [24], [25], [26], [27, 94], [28], [29], [30, 33, 53, 99], [31, 38, 99], [32], [30, 33, 53, 99], [34], [35, 50, 63, 64, 65, 95], [36], [37], [31, 38], [39], [40], [41, 91], [42, 79], [18, 43, 67, 82], [44], [45, 46, 59, 97], [45, 46, 59, 97], [47], [48], [49], [35, 50, 63, 64, 95], [51], [52, 80, 90], [30, 33, 53, 99], [54], [55], [56], [57], [58], [21, 45, 46, 59], [60], [61], [62], [35, 50, 63, 64, 65, 80, 95], [35, 50, 63, 64], [35, 63, 65], [66], [18, 43, 67, 69, 91], [68], [67, 69, 91], [70], [71], [72], [73], [74], [75], [76], [77], [8, 78], [42, 79], [52, 63, 80, 90], [81, 82], [18, 43, 81, 82], [83], [84], [85], [86], [87], [88], [89, 97], [21, 52, 80, 90, 95], [41, 67, 69, 91], [92], [93], [27, 94], [21, 35, 50, 63, 90, 95], [96], [45, 46, 89, 97], [98], [30, 31, 33, 53, 99]]
100 100


## Covering number

### Wasserstein

In [102]:
from mip import Model, maximize, minimize, xsum

In [103]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)
# m.objective = maximize(100 * x + 100 * y)
# 制約条件
for i in range(N):
    b = 0
    for j in nbh_w[i]:
        b += x[j]
    m += b >= 1
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

# print("obj", m.objective.x, "x", x.x, "y", y.x)

15.0


### Regularized Wasserstein

In [104]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)
# m.objective = maximize(100 * x + 100 * y)
# 制約条件
for i in range(N):
    b = 1
    for j in nbh_rw[i]:
        b += x[j]
    m += b >= 2
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

76.0
